In [ ]:
import json
import asyncio
import websockets

In [ ]:
class SimulatorResponder:
    valid_prefix = '42'
    
    def is_valid_message(self, message):
        return message.startswith(self.valid_prefix)
    
    def parse_message(self,message):
        return json.loads(message)
    
    def invoke_pid(self, speed, cte, steering_angle):
        return -0.05
    
    def make_output_string(self, tag, payload):
        return self.valid_prefix + json.dumps([tag, payload])
            
    def process(self, message):
        if self.is_valid_message(message):
            tag, telemetry = self.parse_message(message[2:])
            if tag == 'telemetry':
                steer_value = self.invoke_pid(float(telemetry['speed']),
                                                float(telemetry['cte']),
                                                float(telemetry['steering_angle']))
                output = {
                    'steering_angle': steer_value,
                    'throttle': 0.3 }
                return self.make_output_string('steer', output)
        else:
            return self.make_output_string('manual', {})

In [ ]:
simulator = SimulatorResponder()

async def hello(websocket, path):
    while True:
        message = await websocket.recv()
        response = simulator.process(message)
        await websocket.send(response)
    
start_server = websockets.serve(hello, 'localhost', 4567)
asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()